In [23]:
import math

import numpy as np
from astropy.io import fits
import healpy as hp
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict
import math

def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))


NSIDE = 512
NSIDE_SUB = 2048
NPIX = hp.nside2npix(NSIDE)

In [20]:

decamCCD = fits.open('../../bricks_data/ccds-annotated-decam-dr9.fits')
mosaicCCD = fits.open('../../bricks_data/ccds-annotated-mosaic-dr9.fits')
bassCCD = fits.open('../../bricks_data/ccds-annotated-90prime-dr9.fits')

dataDecam = decamCCD[1].data
dataMosaic = mosaicCCD[1].data
dataBass = bassCCD[1].data

# Extracting systematics
filter_colour = np.concatenate((dataDecam.field('filter'), dataMosaic.field('filter'), dataBass.field('filter')),
                               axis=0)

# camera = np.concatenate((dataDecam.field('camera'), dataMosaic.field('camera'), dataBass.field('camera')), axis=0)

exptime = np.concatenate((dataDecam.field('exptime'), dataMosaic.field('exptime'), dataBass.field('exptime')), axis=0)
airmass = np.concatenate((dataDecam.field('airmass'), dataMosaic.field('airmass'), dataBass.field('airmass')), axis=0)
fwhm = np.concatenate((dataDecam.field('fwhm'), dataMosaic.field('fwhm'), dataBass.field('fwhm')), axis=0)
seeing = fwhm * 0.262
ccdskysb = np.concatenate((dataDecam.field('ccdskysb'), dataMosaic.field('ccdskysb'), dataBass.field('ccdskysb')),
                          axis=0)
ccdskycounts = np.concatenate(
    (dataDecam.field('ccdskycounts'), dataMosaic.field('ccdskycounts'), dataBass.field('ccdskycounts')), axis=0)

# Use this cell to simply import an existing pixel2subpixel mapping


In [21]:
pixel2systematics_dict = defaultdict(list)
samp = 738490

pixels_overall = [samp]
pixel2subpixel_dict = {738490 : [11801320, 11809511, 11809512, 11817703, 11817704, 11817705, 11825894, 11825895, 11825896, 11825897, 11834087, 11834088, 11834089, 11842279, 11842280, 11850472]}
print(pixel2subpixel_dict[samp])

subpixel2ccd_dict = defaultdict(list)
import random
for subpix in pixel2subpixel_dict[samp]:
    subpixel2ccd_dict[subpix] = random.sample(range(1, 6500000), 15)

for subpix in pixel2subpixel_dict[samp]:
    print(subpixel2ccd_dict[subpix])


[11801320, 11809511, 11809512, 11817703, 11817704, 11817705, 11825894, 11825895, 11825896, 11825897, 11834087, 11834088, 11834089, 11842279, 11842280, 11850472]
[3185672, 4823550, 5419531, 3108922, 5096436, 4742303, 5938988, 3317380, 1094459, 802296, 3850448, 5415454, 3609410, 4695080, 4938268]
[2256382, 5604716, 711637, 1303871, 2190138, 4586897, 2085675, 327609, 3433228, 209743, 4653702, 4189956, 2959186, 5961820, 3033603]
[2477462, 6443581, 2028000, 5139118, 3977010, 6228149, 6431372, 3172786, 3357106, 3787551, 4962450, 2107063, 5124803, 1485428, 1317510]
[5771442, 2248204, 5398667, 3336981, 3709556, 3815253, 4302321, 2949243, 417504, 1956499, 3864571, 2893635, 4721086, 5112984, 691281]
[498929, 2632533, 5751313, 1503476, 2057115, 4780172, 4145011, 4926352, 2409242, 770713, 2691293, 4271436, 4890201, 4224248, 1732821]
[5231772, 4855982, 3128599, 6419073, 3460278, 1334539, 6327888, 1860500, 3626190, 4798963, 5346784, 2038911, 2018873, 132515, 3910920]
[4652159, 2804175, 3363169, 3217

In [26]:
for i, sample_pixel in enumerate(pixels_overall):

    subpixels_per_pixel = pixel2subpixel_dict[sample_pixel]

    airmass_mean = airmass_std = airmass_median = 0

    seeing_g_mean = seeing_g_std = seeing_g_median = 0
    seeing_r_mean = seeing_r_std = seeing_r_median = 0
    seeing_z_mean = seeing_z_std = seeing_z_median = 0

    ccdskysb_g_mean = ccdskysb_g_std = ccdskysb_g_median = 0
    ccdskysb_r_mean = ccdskysb_r_std = ccdskysb_r_median = 0
    ccdskysb_z_mean = ccdskysb_z_std = ccdskysb_z_median = 0

    ccdskycounts_g_mean = ccdskycounts_g_std = ccdskycounts_g_median = 0
    ccdskycounts_r_mean = ccdskycounts_r_std = ccdskycounts_r_median = 0
    ccdskycounts_z_mean = ccdskycounts_z_std = ccdskycounts_z_median = 0

    subpixels_covered = subpixels_covered_g = subpixels_covered_r = subpixels_covered_z = 0

    airmass_min = seeing_g_min = seeing_r_min = seeing_z_min = ccdskysb_g_min = ccdskysb_r_min = ccdskysb_z_min = ccdskycounts_g_min = ccdskycounts_r_min = ccdskycounts_z_min = math.inf

    airmass_max = seeing_g_max = seeing_r_max = seeing_z_max = ccdskysb_g_max = ccdskysb_r_max = ccdskysb_z_max = ccdskycounts_g_max = ccdskycounts_r_max = ccdskycounts_z_max = - math.inf

    # Go through all 16 subpixels in the sample pixel
    for subpixel in subpixels_per_pixel:
        # Condition needed in case a subpixel is not covered by CCDs
        if subpixel not in subpixel2ccd_dict.keys():
            continue
        subpixels_covered += 1

        ccds_per_subpixel = subpixel2ccd_dict[subpixel]
        # Get values for singular systematics
        airmass_mean += airmass[ccds_per_subpixel].mean()
        airmass_std += airmass[ccds_per_subpixel].std()
        airmass_median += np.median(airmass[ccds_per_subpixel])
        airmass_min = min(airmass[ccds_per_subpixel].min(), airmass_min)
        airmass_max = max(airmass[ccds_per_subpixel].max(), airmass_max)

        # Get values for per band systematics

        mask_g = (filter_colour[ccds_per_subpixel] == 'g')
        mask_r = (filter_colour[ccds_per_subpixel] == 'r')
        mask_z = (filter_colour[ccds_per_subpixel] == 'z')

        see = seeing[ccds_per_subpixel]
        seeing_g = see[mask_g]
        seeing_r = see[mask_r]
        seeing_z = see[mask_z]


        # Sky background
        sb = ccdskysb[ccds_per_subpixel]
        ccdskysb_g = sb[mask_g]
        ccdskysb_r = sb[mask_r]
        ccdskysb_z = sb[mask_z]

        # Sky Counts
        sc = ccdskycounts[ccds_per_subpixel]
        ccdskycounts_g = sc[mask_g]
        ccdskycounts_r = sc[mask_r]
        ccdskycounts_z = sc[mask_z]

        g_exp = mask_g.sum()
        if g_exp > 0:
            print("Vals in subpixel", seeing_g)
            print("Prev Max", seeing_g_max)
            subpixels_covered_g += 1
            seeing_g_mean += seeing_g.mean()
            seeing_g_std += seeing_g.std()
            seeing_g_median += np.median(seeing_g)
            seeing_g_min = min(seeing_g.min(), seeing_g_min)
            seeing_g_max = max(seeing_g.max() , seeing_g_max)
            print("Post Max", seeing_g_max)
            print()

            ccdskysb_g_mean += ccdskysb_g.mean()
            ccdskysb_g_std += ccdskysb_g.std()
            ccdskysb_g_median += np.median(ccdskysb_g)
            ccdskysb_g_min = min(ccdskysb_g.min(), ccdskysb_g_min)
            ccdskysb_g_max = max(ccdskysb_g.max() , ccdskysb_g_max)

            ccdskycounts_g_mean += ccdskycounts_g.mean()
            ccdskycounts_g_std += ccdskycounts_g.std()
            ccdskycounts_g_median += np.median(ccdskycounts_g)
            ccdskycounts_g_min = min(ccdskycounts_g.min(), ccdskycounts_g_min)
            ccdskycounts_g_max = max(ccdskycounts_g.max(), ccdskycounts_g_max)

        r_exp = mask_r.sum()
        if r_exp > 0:
            subpixels_covered_r += 1
            seeing_r_mean += seeing_r.mean()
            seeing_r_std += seeing_r.std()
            seeing_r_median += np.median(seeing_r)
            seeing_r_min = min(seeing_r.min(),seeing_r_min )
            seeing_r_max = max(seeing_r.max(), seeing_r_max)

            ccdskysb_r_mean += ccdskysb_r.mean()
            ccdskysb_r_std += ccdskysb_r.std()
            ccdskysb_r_median += np.median(ccdskysb_r)
            ccdskysb_r_min = min(ccdskysb_r.min(), ccdskysb_r_min)
            ccdskysb_r_max = max(ccdskysb_r.max(), ccdskysb_r_max)

            ccdskycounts_r_mean += ccdskycounts_r.mean()
            ccdskycounts_r_std += ccdskycounts_r.std()
            ccdskycounts_r_median += np.median(ccdskycounts_r)
            ccdskycounts_r_min = min(ccdskycounts_r.min(),ccdskycounts_r_min )
            ccdskycounts_r_max = max(ccdskycounts_r.max(),ccdskycounts_r_max )

        z_exp = mask_z.sum()
        if z_exp > 0:
            subpixels_covered_z += 1
            seeing_z_mean += seeing_z.mean()
            seeing_z_std += seeing_z.std()
            seeing_z_median += np.median(seeing_z)
            seeing_z_min = min(seeing_z.min(), seeing_z_min)
            seeing_z_max = max(seeing_z.max(), seeing_z_max)

            ccdskysb_z_mean += ccdskysb_z.mean()
            ccdskysb_z_std += ccdskysb_z.std()
            ccdskysb_z_median += np.median(ccdskysb_z)
            ccdskysb_z_min = min(ccdskysb_z.min(),ccdskysb_z_min )
            ccdskysb_z_max = max(ccdskysb_z.max(), ccdskysb_z_max)

            ccdskycounts_z_mean += ccdskycounts_z.mean()
            ccdskycounts_z_std += ccdskycounts_z.std()
            ccdskycounts_z_median += np.median(ccdskycounts_z)
            ccdskycounts_z_min = min(ccdskycounts_z.min(), ccdskycounts_z_min)
            ccdskycounts_z_max = max(ccdskycounts_z.max(), ccdskycounts_z_max)

    if i % 17843 == 0:
        print(int(i / 17843), '%')

    # Do not append to dictionary if none of the subpixels is covered in full
    if subpixels_covered == 0:
        continue

    systematics_per_pixel = []
    systematics_per_pixel.append(airmass_mean / subpixels_covered)
    systematics_per_pixel.append(airmass_std / subpixels_covered)
    systematics_per_pixel.append(airmass_median / subpixels_covered)
    systematics_per_pixel.append(airmass_min)
    systematics_per_pixel.append(airmass_max)

    systematics_per_pixel.append(ccdskysb_g_mean / subpixels_covered_g)
    systematics_per_pixel.append(ccdskysb_g_std / subpixels_covered_g)
    systematics_per_pixel.append(ccdskysb_g_median / subpixels_covered_g)
    systematics_per_pixel.append(ccdskysb_g_min)
    systematics_per_pixel.append(ccdskysb_g_max)

    systematics_per_pixel.append(ccdskysb_r_mean / subpixels_covered_r)
    systematics_per_pixel.append(ccdskysb_r_std / subpixels_covered_r)
    systematics_per_pixel.append(ccdskysb_r_median / subpixels_covered_r)
    systematics_per_pixel.append(ccdskysb_r_min)
    systematics_per_pixel.append(ccdskysb_r_max)

    systematics_per_pixel.append(ccdskysb_z_mean / subpixels_covered_z)
    systematics_per_pixel.append(ccdskysb_z_std / subpixels_covered_z)
    systematics_per_pixel.append(ccdskysb_z_median / subpixels_covered_z)
    systematics_per_pixel.append(ccdskysb_z_min)
    systematics_per_pixel.append(ccdskysb_z_max)

    systematics_per_pixel.append(ccdskycounts_g_mean / subpixels_covered_g)
    systematics_per_pixel.append(ccdskycounts_g_std / subpixels_covered_g)
    systematics_per_pixel.append(ccdskycounts_g_median / subpixels_covered_g)
    systematics_per_pixel.append(ccdskycounts_g_min)
    systematics_per_pixel.append(ccdskycounts_g_max)

    systematics_per_pixel.append(ccdskycounts_r_mean / subpixels_covered_r)
    systematics_per_pixel.append(ccdskycounts_r_std / subpixels_covered_r)
    systematics_per_pixel.append(ccdskycounts_r_median / subpixels_covered_r)
    systematics_per_pixel.append(ccdskycounts_r_min)
    systematics_per_pixel.append(ccdskycounts_r_max)

    systematics_per_pixel.append(ccdskycounts_z_mean / subpixels_covered_z)
    systematics_per_pixel.append(ccdskycounts_z_std / subpixels_covered_z)
    systematics_per_pixel.append(ccdskycounts_z_median / subpixels_covered_z)
    systematics_per_pixel.append(ccdskycounts_z_min)
    systematics_per_pixel.append(ccdskycounts_z_max)

    systematics_per_pixel.append(seeing_g_mean / subpixels_covered_g)
    systematics_per_pixel.append(seeing_g_std / subpixels_covered_g)
    systematics_per_pixel.append(seeing_g_median / subpixels_covered_g)
    systematics_per_pixel.append(seeing_g_min)
    systematics_per_pixel.append(seeing_g_max)

    systematics_per_pixel.append(seeing_r_mean / subpixels_covered_r)
    systematics_per_pixel.append(seeing_r_std / subpixels_covered_r)
    systematics_per_pixel.append(seeing_r_median / subpixels_covered_r)
    systematics_per_pixel.append(seeing_r_min)
    systematics_per_pixel.append(seeing_r_max)

    systematics_per_pixel.append(seeing_z_mean / subpixels_covered_z)
    systematics_per_pixel.append(seeing_z_std / subpixels_covered_z)
    systematics_per_pixel.append(seeing_z_median / subpixels_covered_z)
    systematics_per_pixel.append(seeing_z_min)
    systematics_per_pixel.append(seeing_z_max)

    # Also appending fraction of pixel covered to cut on it later
    systematics_per_pixel.append(subpixels_covered / 16)

    pixel2systematics_dict[sample_pixel] = systematics_per_pixel

Vals in subpixel [1.6717557 1.1307993 1.0107375 1.0523468]
Prev Max -inf
Post Max 1.6717557

Vals in subpixel [1.0425467 1.3244244 1.2235415 1.8993049 1.459865  1.5597082]
Prev Max 1.6717557
Post Max 1.8993049

Vals in subpixel [1.6355538 1.7153108]
Prev Max 1.8993049
Post Max 1.8993049

Vals in subpixel [1.3417366 1.2993284 1.3502179 1.0820057 1.2418855]
Prev Max 1.8993049
Post Max 1.8993049

Vals in subpixel [1.4926134 1.1213332 1.157747  1.2799245]
Prev Max 1.8993049
Post Max 1.8993049

Vals in subpixel [1.2107415 1.3759891 1.4585047 1.6821254 1.4312359]
Prev Max 1.8993049
Post Max 1.8993049

Vals in subpixel [1.9582032 1.2755296 1.7047621 1.2590083 1.2085813 1.2768286]
Prev Max 1.8993049
Post Max 1.9582032

Vals in subpixel [1.1208261 1.1912315 1.309508  1.1616321 1.3961351 1.0631666 1.563898 ]
Prev Max 1.9582032
Post Max 1.9582032

Vals in subpixel [1.146016  1.2410374 1.0252136 1.4201219 1.6313214]
Prev Max 1.9582032
Post Max 1.9582032

Vals in subpixel [1.2645519 1.0365555 1.499

In [27]:
print(pixel2systematics_dict)

defaultdict(<class 'list'>, {738490: [1.3374811485409737, 0.27441586647182703, 1.26749999076128, 1.0, 2.31, 23.41797626018524, 0.28427211102098227, 23.492695569992065, 19.573668, 23.947622, 22.570778369903564, 0.20535148144699633, 22.600833654403687, 21.202356, 23.008324, 20.129544496536255, 0.38582406053319573, 20.164433121681213, 18.742832, 21.006083, 1.8256460800766945, 1.4143272954970598, 1.2174696028232574, 0.7971747, 44.785313, 3.6344459801912308, 0.8554891413077712, 3.399920806288719, 2.2933652, 12.10208, 26.47869575023651, 10.481168538331985, 23.558577299118042, 10.660295, 85.7175, 1.3212026357650757, 0.17742592794820666, 1.2662159502506256, 1.0107375, 1.9582032, 1.2059855163097382, 0.19227653834968805, 1.1747382059693336, 0.7630671, 1.8370883, 1.1032110750675201, 0.17190447403118014, 1.0864915400743484, 0.6807407, 1.7637646, 1.0]})
